# Import library

In [1]:
import pandas as pd
import yaml

# Variable config

## Read yaml file 

In [2]:
with open('../config.yaml', 'r') as file:
    config = yaml.safe_load(file)

In [3]:
data_path = config['DATA_PATH'] ## '../data/dress_rental_prices news.csv'

# Read data

In [4]:
df = pd.read_csv(data_path)

In [5]:
df.head()

,numbers,ID,Name,Brand,Colour,Catagories,Price
0,0,74416,Runway stripe dress,Stella McCartney,beige,dresses,111
1,1,73815,Reformation Kourtney Dress,Reformation,beige,dresses,50
2,2,73801,Ivory Viola bridal dress,Ghost,beige,dresses,75
3,3,73718,Pasu Dress - Rhino Tusk,Coucoo,beige,dresses,37
4,4,73605,Ellen,RIXO,beige,dresses,47


# Explore data

## data type

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29997 entries, 0 to 29996
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   numbers     29997 non-null  int64 
 1   ID          29997 non-null  int64 
 2   Name        29995 non-null  object
 3   Brand       29997 non-null  object
 4   Colour      29997 non-null  object
 5   Catagories  29997 non-null  object
 6   Price       29997 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 1.6+ MB


## describe

In [7]:
df.describe(percentiles=[.1, .90, .95, .99])

,numbers,ID,Price
count,29997.000000,29997.000000,29997.000000
mean,14998.000000,42548.336400,61.668800
std,8659.532349,19867.282746,38.138688
min,0.000000,122.000000,11.000000
10%,2999.600000,14777.600000,32.000000
50%,14998.000000,42902.000000,51.000000
90%,26996.400000,68848.400000,102.000000
95%,28496.200000,71589.200000,139.000000
99%,29696.040000,73993.120000,195.000000
max,29996.000000,75024.000000,793.000000


## ydata-profiling

In [8]:
## let see in exlore_data.ipynb

## nan value

In [9]:
df.isna().sum()

numbers       0
ID            0
Name          2
Brand         0
Colour        0
Catagories    0
Price         0
dtype: int64

# Transform data

## drop nan value

In [10]:
df = df[~df['Name'].isna()]

## drop numbers column

In [11]:
df.drop('numbers', axis= 1 , inplace = True)

## change data type

In [12]:
df['ID'] = df['ID'].astype('object')
df['Brand'] = df['Brand'].astype('category')
df['Colour'] = df['Colour'].astype('category')

## split categories and replace nan with no-sub-category

In [13]:
sub_categories = df['Catagories'].str.split(',', expand=True)
sub_categories = sub_categories.fillna('no-sub-categories')
sub_categories.columns = [f'sub-Catagories-{i+1}' for i in range(sub_categories.shape[1])]

df = pd.concat([df, sub_categories], axis=1)
df = df.drop(columns=['Catagories'])

## filter data with less than 400

In [14]:
df = df[df['Price'] < 400]

# Feature engineering

## Calculate average price by brand and add as a new column

In [15]:
df['Avg_Price_By_Brand'] = df.groupby('Brand')['Price'].transform('mean')

C:\Users\Rattapon.San\AppData\Local\Temp\ipykernel_30048\1663022941.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['Avg_Price_By_Brand'] = df.groupby('Brand')['Price'].transform('mean')


## Calculate average price by (brand, colour) and add as a new column

In [16]:
df['Avg_Price_By_Brand_Colour'] = df.groupby(['Brand', 'Colour'])['Price'].transform('mean')

C:\Users\Rattapon.San\AppData\Local\Temp\ipykernel_30048\3750462187.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['Avg_Price_By_Brand_Colour'] = df.groupby(['Brand', 'Colour'])['Price'].transform('mean')


## Create a new column to count the number of sub-categories that are not 'no-sub-categories'

In [17]:
df['Category_Count'] = (sub_categories != 'no-sub-categories').sum(axis=1)

In [18]:
df

,ID,Name,Brand,Colour,Price,sub-Catagories-1,sub-Catagories-2,sub-Catagories-3,sub-Catagories-4,sub-Catagories-5,...,sub-Catagories-20,sub-Catagories-21,sub-Catagories-22,sub-Catagories-23,sub-Catagories-24,sub-Catagories-25,sub-Catagories-26,Avg_Price_By_Brand,Avg_Price_By_Brand_Colour,Category_Count
0,74416,Runway stripe dress,Stella McCartney,beige,111,dresses,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,...,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,127.189189,109.500000,1
1,73815,Reformation Kourtney Dress,Reformation,beige,50,dresses,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,...,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,50.490328,53.534884,1
2,73801,Ivory Viola bridal dress,Ghost,beige,75,dresses,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,...,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,41.660900,40.117647,1
3,73718,Pasu Dress - Rhino Tusk,Coucoo,beige,37,dresses,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,...,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,44.692308,37.250000,1
4,73605,Ellen,RIXO,beige,47,dresses,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,...,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,51.809540,51.790323,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29992,872,Silk and wool-blend crepe dress yellow,Victoria Beckham,yellow,102,dresses,Smart,midi,Short Sleeve,Pencil,...,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,116.462963,102.000000,6
29993,871,La Guardia Matching Set,Faithfull The Brand,yellow,43,dresses,Floral,Polka Dot,Loose,Smart Casual,...,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,38.698690,35.088235,11
29994,844,One-shoulder ruffled crepe mini dress,Solace London,yellow,75,dresses,Vinyl-PVC-Synthetics,Smart,Formal,Winter,...,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,79.911565,65.250000,8
29995,592,Billie Dress,Rebecca Vallance,yellow,42,dresses,Cotton-Jersey,Tie-Front,Skinny-Fit,Smart,...,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,no-sub-categories,82.153846,64.500000,10
